In [14]:
# Importation des modules

import pandas as pd
from gensim.models import Word2Vec
import random
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import datetime

#Affichage de toutes les colonnes

pd.set_option('display.max_columns', 500)

In [15]:
w2v_model_all = Word2Vec.load('results/word2vec_all_300.model')

In [16]:
df0 = pd.read_csv("data/data_cleaned_NLP.csv", sep = ',', encoding = 'latin-1')

In [17]:
df = df0[['Date', 'Groupe', 'Orateur', 'tokenized_replique']].copy()

In [18]:
df['days'] = df.apply(lambda row : (datetime.date(int(row.Date[:4]),int(row.Date[5:7]),int(row.Date[8:])) - 
                    datetime.date(2017,6,28)).days, axis = 1)

In [19]:
def add_groupe(L, etiquette):
    return [i + etiquette for i in L]

In [11]:
def calcule_WMD_intervalle(d1, d2):
    print('JOURS : ', d1, ' ', d2)
    df1 = df[df['days'] >= d1]
    df1 = df1[df1['days'] < d2]
    
    df_Novice = df1[df1['Groupe'] == 'Novice']
    df_Exp = df1[df1['Groupe'] == 'Exp']
    
    #display(df_Novice)
    try:
        df_Novice = df1[df1['Groupe'] == 'Novice'].sample(50).dropna()
        df_Exp = df1[df1['Groupe'] == 'Exp'].sample(50).dropna()
        print(df_Exp.shape, df_Novice.shape)
    except:
        return np.nan
    
    phrases_Exp = [df_Exp.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Exp.shape[0])]
    phrases_Novice = [df_Novice.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Novice.shape[0])]
    
    WMD = []
    

    for i in phrases_Novice:
        value = np.mean([w2v_model_all.wv.wmdistance(add_groupe(i, '_Novice'), 
                                                     add_groupe(j, '_Exp')) for j in phrases_Exp])
        print(value)
        if str(value) == 'inf':
            print(i)
        WMD.append(value)
    
    px.line([np.mean(WMD[:i]) for i in range(1, len(WMD))]).show()
    
    return np.mean(WMD)

In [140]:
#calcule_WMD_intervalle(322, 336)

In [141]:
#L = [calcule_WMD_intervalle(i, i + 14) for i in range(0, 1094, 14)]
#px.line(y = L, x = [i for i in range(0, 1094, 14)])

values = list((pd.read_csv('results/WMDinter_temporel.csv')['0']))
dates = list((pd.read_csv('results/WMDinter_temporel.csv')['Unnamed: 0']))

In [142]:
#pd.DataFrame(L, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel0.csv')

In [143]:
px.line(y = values, x = dates)

In [158]:
date_label = [datetime.timedelta(days=i) + datetime.date(2017,6,28) for i in range(14, 1094, 14)]

In [159]:
px.line(y = [np.mean(values[:i]) for i in range(1,len(values))], x = date_label, 
        labels = {'x' : 'Dates', 'y' : 'Moyenne cumulée des distances'})

On observe la moyenne cumulée des distances en fonction du temps. On constate une moyenne forte en début d'exercice, puis une relaxation pour atteindre une valeur limite.
On peut interpréter cela comme : 
- un début d'exercice où les novices et les expérimentés parlent différemment, la distance entre les deux langages est raltivement grande.
- une période d'apprentissage (du 20 septembre 2017 au 16 mai 2018) où les différences entre novices et expérimentés tendent à s'estomper (mais dans quel sens ? on s'attend à ce que les novices s'adaptent aux expérimentés, et pas forcement l'inverse), la distance diminue.
- une période de différenciation (du 16 mai 2018 au 12 décembre 2018), où la distance augmente légerement : comme si un des deux camps voulait se différencier de l'autre.
- une période stable (du 12 décembre 2018 à la fin de l'exercice).

## Analyse personnalisée député par député

In [8]:
# Députés Novices parlant le plus (classement par nombre d'intervention)

dict(df0[df0['Groupe'] == 'Novice'].Orateur.value_counts())

{'laurent pietraszewski': 3435,
 'francois ruffin': 3125,
 'sabine rubin': 2189,
 'adrien taquet': 1781,
 'roland lescure': 1705,
 'laetitia avia': 1637,
 'emilie chalas': 1579,
 'cendra motin': 1557,
 'yael braun pivet': 1286,
 'catherine fabre': 1159,
 'coralie dubost': 1138,
 'christophe naegelen': 1134,
 'thomas mesnier': 1121,
 'jean baptiste djebbari': 1096,
 'nadia hai': 986,
 'marie christine verdier jouclas': 986,
 'bruno studer': 895,
 'sacha houlie': 880,
 'gilles le gendre': 836,
 'berangere couillard': 714,
 'veronique riotton': 675,
 'benedicte peyrol': 603,
 'alexandra louis': 603,
 'raphael gauvain': 598,
 'raphael gerard': 594,
 'mohamed laqhila': 586,
 'agnes thill': 568,
 'martine wonner': 542,
 'cecile rilhac': 527,
 'stephanie rist': 478,
 'zivka park': 469,
 'perrine goulet': 461,
 'vincent thiebaut': 446,
 'jean francois mbaye': 439,
 'alice thourot': 391,
 'alexandre holroyd': 361,
 'caroline abadie': 336,
 'annie chapelier': 326,
 'fiona lazaar': 322,
 'damien 

In [9]:
orateurs_novices = list(dict(df0[df0['Groupe'] == 'Novice'].Orateur.value_counts()).keys())

In [11]:
def calcule_WMD_intervalle_personnalise(orateur, d1, d2):
    print(orateur)
    print('JOURS : ', d1, ' ', d2)
    df1 = df[df['days'] >= d1]
    df1 = df1[df1['days'] < d2]
    
    df_Novice = df1[df1['Orateur'] == orateur]
    df_Exp = df1[df1['Groupe'] == 'Exp']
    
    #display(df_Novice)
    try:
        df_Novice = df1[df1['Groupe'] == 'Novice'].dropna().sample(50)
        df_Exp = df1[df1['Groupe'] == 'Exp'].dropna().sample(50)
    except:
        return np.nan
    
    phrases_Exp = [df_Exp.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Exp.shape[0])]
    phrases_Novice = [df_Novice.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Novice.shape[0])]
    
    WMD = []
    
    for i in phrases_Novice:
        if len(i)<3:
            continue
        values = []
        for j in phrases_Exp:
            d = w2v_model_all.wv.wmdistance(add_groupe(i, '_Novice'), 
                                                     add_groupe(j, '_Exp'))
            if str(d) != 'inf':
                values.append(d)
        value = np.mean(values)
        #print(value)
        WMD.append(value)
    
    #px.line([np.mean(WMD[:i]) for i in range(1, len(WMD))]).show()
    print(np.mean(WMD))
    
    return np.mean(WMD)

### 1) Pietraszewski

In [244]:
#L_pietraszewski = [calcule_WMD_intervalle_personnalise('laurent pietraszewski', i, i + 14) for i in range(0, 1094, 14)]
#pd.DataFrame(L_pietraszewski, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel_pietraszewski.csv')

In [242]:
px.line([np.mean(L_pietraszewski[:i]) for i in range(1,len(L_pietraszewski))])

### 2) Ruffin

In [266]:
#L_ruffin = [calcule_WMD_intervalle_personnalise('francois ruffin', i, i + 14) for i in range(0, 1094, 14)]
#pd.DataFrame(L_ruffin, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel_ruffin.csv')

In [265]:
px.line([np.mean(L_ruffin[:i]) for i in range(1,len(L_ruffin))])

In [245]:
for i in range(len(L_ruffin)):
    if str(L_ruffin[i]) == 'nan':
        L_ruffin[i] = L_ruffin[i-1]
    elif str(L_ruffin[i]) == 'inf':
        print(i, [(j, j+14) for j in range(0, 1094, 14)][i])

8 (112, 126)
27 (378, 392)
33 (462, 476)
42 (588, 602)
51 (714, 728)
73 (1022, 1036)


In [ ]:
L_ruffin

#### Autres députés

In [29]:
orateurs_novices

['laurent pietraszewski',
 'francois ruffin',
 'sabine rubin',
 'adrien taquet',
 'roland lescure',
 'laetitia avia',
 'emilie chalas',
 'cendra motin',
 'yael braun pivet',
 'catherine fabre',
 'coralie dubost',
 'christophe naegelen',
 'thomas mesnier',
 'jean baptiste djebbari',
 'nadia hai',
 'marie christine verdier jouclas',
 'bruno studer',
 'sacha houlie',
 'gilles le gendre',
 'berangere couillard',
 'veronique riotton',
 'benedicte peyrol',
 'alexandra louis',
 'raphael gauvain',
 'raphael gerard',
 'mohamed laqhila',
 'agnes thill',
 'martine wonner',
 'cecile rilhac',
 'stephanie rist',
 'zivka park',
 'perrine goulet',
 'vincent thiebaut',
 'jean francois mbaye',
 'alice thourot',
 'alexandre holroyd',
 'caroline abadie',
 'annie chapelier',
 'fiona lazaar',
 'damien adam',
 'albane gaillot',
 'dominique david',
 'fannette charvier',
 'gregory besson moreau',
 'anne genetet',
 'typhanie degois',
 'joachim son forget',
 'audrey dufeu schubert',
 'sylvie charriere',
 'carole

In [21]:
for orateur in orateurs_novices[2:11]:
    T = [calcule_WMD_intervalle_personnalise(orateur, i, i + 14) for i in range(0, 1094, 14)]
    pd.DataFrame(T, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel_' + orateur + '.csv')


sabine rubin
JOURS :  0   14
1.5097795256273057
sabine rubin
JOURS :  14   28
1.5216195955797323
sabine rubin
JOURS :  28   42
1.504244272763923
sabine rubin
JOURS :  42   56
1.5171095398136683
sabine rubin
JOURS :  56   70
sabine rubin
JOURS :  70   84
sabine rubin
JOURS :  84   98
1.4996234188927395
sabine rubin
JOURS :  98   112
1.4945687556293403
sabine rubin
JOURS :  112   126
1.4988918014560784
sabine rubin
JOURS :  126   140
1.4966546735627992
sabine rubin
JOURS :  140   154
1.5044590476211976
sabine rubin
JOURS :  154   168
1.5097408540014166
sabine rubin
JOURS :  168   182
1.5027361980055174
sabine rubin
JOURS :  182   196
sabine rubin
JOURS :  196   210
1.5046315219535675
sabine rubin
JOURS :  210   224
1.4940503015752984
sabine rubin
JOURS :  224   238
1.503509713265083
sabine rubin
JOURS :  238   252
1.47930231111796
sabine rubin
JOURS :  252   266
1.4864011761595002
sabine rubin
JOURS :  266   280
1.4950919663443656
sabine rubin
JOURS :  280   294
1.5102935602323933
sabine

1.5014468035119488
roland lescure
JOURS :  154   168
1.5024435731192614
roland lescure
JOURS :  168   182
1.497710884196778
roland lescure
JOURS :  182   196
roland lescure
JOURS :  196   210
1.4933983195716263
roland lescure
JOURS :  210   224
1.4979511536689778
roland lescure
JOURS :  224   238
1.4958391850656374
roland lescure
JOURS :  238   252
1.5032928573415294
roland lescure
JOURS :  252   266
1.4848004180745042
roland lescure
JOURS :  266   280
1.502217817847896
roland lescure
JOURS :  280   294
1.5046201106045287
roland lescure
JOURS :  294   308
1.497361442907041
roland lescure
JOURS :  308   322
1.4833728003725961
roland lescure
JOURS :  322   336
1.5030795614827457
roland lescure
JOURS :  336   350
1.4932575294815222
roland lescure
JOURS :  350   364
1.505878737244407
roland lescure
JOURS :  364   378
1.4975671201219598
roland lescure
JOURS :  378   392
1.509563998754371
roland lescure
JOURS :  392   406
1.5020768855079039
roland lescure
JOURS :  406   420
roland lescure
JO

1.4769741228012758
emilie chalas
JOURS :  252   266
1.4920531109006865
emilie chalas
JOURS :  266   280
1.507034087922176
emilie chalas
JOURS :  280   294
1.505348635738366
emilie chalas
JOURS :  294   308
1.5065473765882953
emilie chalas
JOURS :  308   322
1.5108505837687005
emilie chalas
JOURS :  322   336
1.4980194452315942
emilie chalas
JOURS :  336   350
1.5035307461879757
emilie chalas
JOURS :  350   364
1.5184708953424095
emilie chalas
JOURS :  364   378
1.49653140828393
emilie chalas
JOURS :  378   392
1.497319148196959
emilie chalas
JOURS :  392   406
1.5006682351585927
emilie chalas
JOURS :  406   420
emilie chalas
JOURS :  420   434
emilie chalas
JOURS :  434   448
1.4963206893984542
emilie chalas
JOURS :  448   462
1.4974062915103066
emilie chalas
JOURS :  462   476
1.5079226503764096
emilie chalas
JOURS :  476   490
1.5091362456521347
emilie chalas
JOURS :  490   504
1.4930334261688891
emilie chalas
JOURS :  504   518
1.5169876691047939
emilie chalas
JOURS :  518   532
1.4

/Users/lilian/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning:

Mean of empty slice.

/Users/lilian/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars



nan
cendra motin
JOURS :  392   406
1.5067554095413882
cendra motin
JOURS :  406   420
cendra motin
JOURS :  420   434
cendra motin
JOURS :  434   448
1.5055556086798396
cendra motin
JOURS :  448   462
1.5096962710756645
cendra motin
JOURS :  462   476
1.5064623852121612
cendra motin
JOURS :  476   490
1.4983828631841696
cendra motin
JOURS :  490   504
1.4937227693615232
cendra motin
JOURS :  504   518
1.519463451709069
cendra motin
JOURS :  518   532
1.5030508610712883
cendra motin
JOURS :  532   546
1.5030858910343776
cendra motin
JOURS :  546   560
cendra motin
JOURS :  560   574
1.5009425425437142
cendra motin
JOURS :  574   588
1.4999308708515884
cendra motin
JOURS :  588   602
1.503779668986223
cendra motin
JOURS :  602   616
1.4988461814857406
cendra motin
JOURS :  616   630
1.5038525792946507
cendra motin
JOURS :  630   644
1.4916270657286446
cendra motin
JOURS :  644   658
1.4893324545990974
cendra motin
JOURS :  658   672
1.492445516105002
cendra motin
JOURS :  672   686
1.49

1.5055334354027892
catherine fabre
JOURS :  448   462
1.4956000609645208
catherine fabre
JOURS :  462   476
1.5051627914410624
catherine fabre
JOURS :  476   490
1.506507971332629
catherine fabre
JOURS :  490   504
1.4990249980744232
catherine fabre
JOURS :  504   518
1.5066704114935434
catherine fabre
JOURS :  518   532
1.5004171971633984
catherine fabre
JOURS :  532   546
1.5015842684587601
catherine fabre
JOURS :  546   560
catherine fabre
JOURS :  560   574
1.5048299059353194
catherine fabre
JOURS :  574   588
1.502058866015407
catherine fabre
JOURS :  588   602
1.4928401130046358
catherine fabre
JOURS :  602   616
1.4923155444424554
catherine fabre
JOURS :  616   630
1.5073765744428678
catherine fabre
JOURS :  630   644
1.4914982085601678
catherine fabre
JOURS :  644   658
1.5019094872518775
catherine fabre
JOURS :  658   672
1.4924537773774826
catherine fabre
JOURS :  672   686
1.504209243918797
catherine fabre
JOURS :  686   700
1.5019518620046644
catherine fabre
JOURS :  700   

In [ ]:
for orateur in orateurs_novices[11:]:
    T = [calcule_WMD_intervalle_personnalise(orateur, i, i + 14) for i in range(0, 1094, 14)]
    pd.DataFrame(T, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel_' + orateur + '.csv')

v

christophe naegelen
JOURS :  0   14
1.5041264509327203
christophe naegelen
JOURS :  14   28
1.5009406989726606
christophe naegelen
JOURS :  28   42
1.5106056689062846
christophe naegelen
JOURS :  42   56
1.5043130283141943
christophe naegelen
JOURS :  56   70
christophe naegelen
JOURS :  70   84
christophe naegelen
JOURS :  84   98
1.4988839678280543
christophe naegelen
JOURS :  98   112
1.5139772824082334
christophe naegelen
JOURS :  112   126
1.4920527786505955
christophe naegelen
JOURS :  126   140
1.493121309234117
christophe naegelen
JOURS :  140   154
1.497687642762507
christophe naegelen
JOURS :  154   168
1.4951379920735854
christophe naegelen
JOURS :  168   182
1.4959976439896
christophe naegelen
JOURS :  182   196
christophe naegelen
JOURS :  196   210
1.4931075410619798
christophe naegelen
JOURS :  210   224
1.501020286372129
christophe naegelen
JOURS :  224   238
1.5026440797746974
christophe naegelen
JOURS :  238   252
1.4906567616757203
christophe naegelen
JOURS :  252   

1.5004437731089066
thomas mesnier
JOURS :  1078   1092


/Users/lilian/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning:

Mean of empty slice.

/Users/lilian/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars



nan
thomas mesnier
JOURS :  1092   1106
1.500756636610909
jean baptiste djebbari
JOURS :  0   14
1.5067889406851027
jean baptiste djebbari
JOURS :  14   28
1.5145904886127315
jean baptiste djebbari
JOURS :  28   42
1.508759996470925
jean baptiste djebbari
JOURS :  42   56
1.4989366302996785
jean baptiste djebbari
JOURS :  56   70
jean baptiste djebbari
JOURS :  70   84
jean baptiste djebbari
JOURS :  84   98
1.4981252445576965
jean baptiste djebbari
JOURS :  98   112
1.508669561731922
jean baptiste djebbari
JOURS :  112   126
1.4938652095319982
jean baptiste djebbari
JOURS :  126   140
1.500182162230248
jean baptiste djebbari
JOURS :  140   154
1.5031154228860044
jean baptiste djebbari
JOURS :  154   168
1.4978909023691156
jean baptiste djebbari
JOURS :  168   182
1.5010949200908044
jean baptiste djebbari
JOURS :  182   196
jean baptiste djebbari
JOURS :  196   210
1.5034687959628303
jean baptiste djebbari
JOURS :  210   224
1.4984927105008454
jean baptiste djebbari
JOURS :  224   238


1.5094412036147253
marie christine verdier jouclas
JOURS :  0   14
1.4919787712020938
marie christine verdier jouclas
JOURS :  14   28
1.5126180745557178
marie christine verdier jouclas
JOURS :  28   42
1.5182677092265848
marie christine verdier jouclas
JOURS :  42   56
1.494771818331527
marie christine verdier jouclas
JOURS :  56   70
marie christine verdier jouclas
JOURS :  70   84
marie christine verdier jouclas
JOURS :  84   98
1.5063606259695348
marie christine verdier jouclas
JOURS :  98   112
1.4971772861259227
marie christine verdier jouclas
JOURS :  112   126
1.4988938482944631
marie christine verdier jouclas
JOURS :  126   140
1.4989472784494056
marie christine verdier jouclas
JOURS :  140   154
1.5135506636884795
marie christine verdier jouclas
JOURS :  154   168
1.4900272138737272
marie christine verdier jouclas
JOURS :  168   182
1.495490298202946
marie christine verdier jouclas
JOURS :  182   196
marie christine verdier jouclas
JOURS :  196   210
1.4907416681844579
marie 

1.490621512218354
bruno studer
JOURS :  854   868
1.491881706143678
bruno studer
JOURS :  868   882
1.4958263985161337
bruno studer
JOURS :  882   896
1.4977404312118576
bruno studer
JOURS :  896   910
1.500233245848476
bruno studer
JOURS :  910   924
1.5007167348403678
bruno studer
JOURS :  924   938
1.492220403348434
bruno studer
JOURS :  938   952
1.4965838408996328
bruno studer
JOURS :  952   966
1.5034931902494428
bruno studer
JOURS :  966   980
1.5099439554918772
bruno studer
JOURS :  980   994
1.5017558725859317
bruno studer
JOURS :  994   1008
1.5022538896538844
bruno studer
JOURS :  1008   1022
bruno studer
JOURS :  1022   1036
1.4972813288889852
bruno studer
JOURS :  1036   1050
1.499943401813084
bruno studer
JOURS :  1050   1064
1.4891058207669658
bruno studer
JOURS :  1064   1078
1.4968055917076732
bruno studer
JOURS :  1078   1092
1.496089437895457
bruno studer
JOURS :  1092   1106
1.4986011383433606
sacha houlie
JOURS :  0   14
1.5071427950585987
sacha houlie
JOURS :  14 

/Users/lilian/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning:

Mean of empty slice.

/Users/lilian/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars



nan
gilles le gendre
JOURS :  896   910
1.5022269257320668
gilles le gendre
JOURS :  910   924
1.4881765244370984
gilles le gendre
JOURS :  924   938
1.4962008351449063
gilles le gendre
JOURS :  938   952
1.4981997701547802
gilles le gendre
JOURS :  952   966
1.5065813615574914
gilles le gendre
JOURS :  966   980
1.5021141205342843
gilles le gendre
JOURS :  980   994
1.5017451244869378
gilles le gendre
JOURS :  994   1008
1.504809644161299
gilles le gendre
JOURS :  1008   1022
gilles le gendre
JOURS :  1022   1036
1.4991198659984724
gilles le gendre
JOURS :  1036   1050
1.5142647299881478
gilles le gendre
JOURS :  1050   1064
1.4923190725618498
gilles le gendre
JOURS :  1064   1078
1.4829136315841471
gilles le gendre
JOURS :  1078   1092
1.5013808448021708
gilles le gendre
JOURS :  1092   1106
1.511114527745032
berangere couillard
JOURS :  0   14
1.5035399216069043
berangere couillard
JOURS :  14   28
1.5173781781753806
berangere couillard
JOURS :  28   42
1.5034432095773684
berangere 

## Nombre de mots utilisés selon la période

In [ ]:
def calcule_nb_mots_periode(d1, d2):
    print('JOURS : ', d1, ' ', d2)
    df1 = df[df['days'] >= d1]
    df1 = df1[df1['days'] < d2]
    
    df_Novice = df1[df1['Orateur'] == 'Novice']
    
    
    
    
    #display(df_Novice)
    try:
        df_Novice = df1[df1['Groupe'] == 'Novice'].dropna().sample(50)
        df_Exp = df1[df1['Groupe'] == 'Exp'].dropna().sample(50)
        print(df_Exp.shape, df_Novice.shape)
    except:
        return np.nan
    
    phrases_Exp = [df_Exp.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Exp.shape[0])]
    phrases_Novice = [df_Novice.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Novice.shape[0])]
    
    WMD = []
    

    for i in phrases_Novice:
        value = np.mean([w2v_model_all.wv.wmdistance(add_groupe(i, '_Novice'), 
                                                     add_groupe(j, '_Exp')) for j in phrases_Exp])
        print(value)
        if str(value) == 'inf':
            print(i)
        WMD.append(value)
    
    px.line([np.mean(WMD[:i]) for i in range(1, len(WMD))]).show()
    
    return np.mean(WMD)